In [1]:
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

engine = create_engine('sqlite+pysqlite:///app/db/data.db', echo=True)

In [2]:
from sqlalchemy import select

from app.models.item import Item
from app.models.stock import Stock
from app.models.cat import Cat
from app.dao.db_base import Base

session = Session(engine)

In [3]:
stmt = (
    select(Item, Stock, Cat)
    .join(Item.stock)
    .join(Item.cat)
    .where(Stock.ready == False)
)

In [4]:
print(stmt)

SELECT items.stock_id, items.load_id, items.cat_id, items.lot, items.pallet, items.roll, items.note, items.id, stocks.reference, stocks.date, stocks.consignor, stocks.ready, stocks.note AS note_1, stocks.id AS id_1, cats.name, cats.cat, cats.width, cats.weight, cats.note AS note_2, cats.id AS id_2 
FROM items JOIN stocks ON stocks.id = items.stock_id JOIN cats ON cats.id = items.cat_id 
WHERE stocks.ready = false


In [5]:
result = session.execute(stmt).scalars().all()

2025-04-17 21:15:21,957 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-17 21:15:21,961 INFO sqlalchemy.engine.Engine SELECT items.stock_id, items.load_id, items.cat_id, items.lot, items.pallet, items.roll, items.note, items.id, stocks.reference, stocks.date, stocks.consignor, stocks.ready, stocks.note AS note_1, stocks.id AS id_1, cats.name, cats.cat, cats.width, cats.weight, cats.note AS note_2, cats.id AS id_2 
FROM items JOIN stocks ON stocks.id = items.stock_id JOIN cats ON cats.id = items.cat_id 
WHERE stocks.ready = 0
2025-04-17 21:15:21,963 INFO sqlalchemy.engine.Engine [generated in 0.00130s] ()


In [6]:
from app.schemas.item import ItemInStock
items_on_stock = [ItemInStock.model_validate(item) for item in result]

2025-04-17 21:15:21,995 INFO sqlalchemy.engine.Engine SELECT stocks.reference AS stocks_reference, stocks.date AS stocks_date, stocks.consignor AS stocks_consignor, stocks.ready AS stocks_ready, stocks.note AS stocks_note, stocks.id AS stocks_id 
FROM stocks 
WHERE stocks.id = ?
2025-04-17 21:15:21,996 INFO sqlalchemy.engine.Engine [generated in 0.00086s] (1,)
2025-04-17 21:15:21,997 INFO sqlalchemy.engine.Engine SELECT cats.name AS cats_name, cats.cat AS cats_cat, cats.width AS cats_width, cats.weight AS cats_weight, cats.note AS cats_note, cats.id AS cats_id 
FROM cats 
WHERE cats.id = ?
2025-04-17 21:15:21,998 INFO sqlalchemy.engine.Engine [generated in 0.00038s] (3,)
2025-04-17 21:15:21,999 INFO sqlalchemy.engine.Engine SELECT stocks.reference AS stocks_reference, stocks.date AS stocks_date, stocks.consignor AS stocks_consignor, stocks.ready AS stocks_ready, stocks.note AS stocks_note, stocks.id AS stocks_id 
FROM stocks 
WHERE stocks.id = ?
2025-04-17 21:15:22,000 INFO sqlalchemy.

In [7]:
items_on_stock

[ItemInStock(id=1, lot='00245', pallet='0034', roll='0101', note='Item note', stock=Stock(reference='FEU034/25', date=datetime.datetime(2025, 2, 2, 0, 0), consignor='Pujol', ready=False, note='tech note'), cat=Cat(name='Visual', cat='film', width=1800, weight=123.0, note='Сгенерированное примечание к категории')),
 ItemInStock(id=2, lot='00245', pallet='0034', roll='0103', note='Item note', stock=Stock(reference='FEU036/25', date=datetime.datetime(2025, 2, 10, 0, 0), consignor='Pujol', ready=False, note='tech note'), cat=Cat(name='Visual', cat='film', width=1800, weight=123.0, note='Сгенерированное примечание к категории')),
 ItemInStock(id=3, lot='00430', pallet='0055', roll='1022', note='Item note', stock=Stock(reference='FEU034/25', date=datetime.datetime(2025, 2, 2, 0, 0), consignor='Pujol', ready=False, note='tech note'), cat=Cat(name='Visual', cat='film', width=2200, weight=156.0, note='Сгенерированное примечание к категории')),
 ItemInStock(id=4, lot='20003', pallet='0204', roll